In [236]:
from google import google
import pandas as pd
from time import sleep
import csv
import pickle
import re
import spacy
import numpy as np
from sklearn.preprocessing import StandardScaler

In [275]:
#load text example
posts = pd.read_csv('./Data/sample_5000.csv')
posts['text'][400:410]

400    WOW! ITS ALMOST 4 IN THE AFTERNOON AND I HAVEN...
401    U know what i cant understand why the weather ...
402    ok people i don't want to join skiing this yea...
403    Peace.Love.Happiness...Finding peace is easy.....
404    says well my bed is calling me, even though 3 ...
405    If there's a possibility you might want me and...
406    When the Nazis came for the communists, I rema...
407    Glad some of the people in my lfe inspired me ...
408    left boston for new york at around 11am. still...
409    So yes i need to lose 75 pounds to get to my i...
Name: text, dtype: object

In [136]:
# load spacy object
nlp = spacy.load('en')

In [137]:
#tokenize
#remove punctionations
def preprocess(sent):
    #remove punctustion
    sent = re.sub(r'[^\w\s]','',sent)
    words = sent.split()
    new_words = []
    for w in words:      
        new_words.append(w.lower())
        
    return ' '.join(new_words)

In [271]:
class mySearchRes:
    def __init__(self,text):
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
        self.cos = []

    def __hash__(self):
        return self.quoteID


# def search_google(row):
#     objects = {}
#     #for row in file:
#     texthash = hash(row)
#     search_results = google.search(row, 1)
#     if texthash not in objects:
#         #print(row)
#         objects[texthash] = mySearchRes(row)       
#         #search_results = [[text, i.link, i.name,i.description] for i in google.search(text, num_page)]
#         for i in search_results:
#             objects[texthash].name.append(i.name)
#             objects[texthash].description.append(i.description)
#             line = preprocess(i.name)
#             count = line.count("lyric") + line.count("lyrics") + line.count("quote") + line.count("quotes")
#             objects[texthash].scores.append(count)
#     return objects

def search_google_DF(file):
    objects = {}
    for row in file:
        texthash = hash(row)
        search_results = google.search(row, 1)
        if texthash not in objects:
            #print(row)
            objects[texthash] = mySearchRes(row)       
            #search_results = [[text, i.link, i.name,i.description] for i in google.search(text, num_page)]
            for i in search_results:
                objects[texthash].name.append(i.name)
                objects[texthash].description.append(i.description)
                line = preprocess(i.name)
                count = line.count("lyric") + line.count("lyrics") + line.count("quote") + line.count("quotes")
                objects[texthash].scores.append(count)
    return objects

def cosineSim(results):
    for item in results:
        str1 = ''.join (results[item].description)
        doc2 = nlp(preprocess(str1))
        doc1 = nlp(preprocess(results[item].quoteText))
        results[item].cos.append(doc1.similarity(doc2))

#pack the score to 10
def align_scores(objects):
    for item in objects:
        count = 0
        for i in range(len(objects[item].scores)): # add 1 to all the scores 
            objects[item].scores[i] = objects[item].scores[i] +1           
            if objects[item].scores[i] is not None:
                count = count + 1 
               # score = score + 1 won't work because score is a shallow copy
        #print(objects[item].scores)
        while count < 10:
            objects[item].scores.append(0) #empty result is replaced by 0
            count = count + 1

def append_features(ob):
    proto_matrix = []
    for item in ob:
        col2 = np.append(ob[item].scores, ob[item].cos)
        #print(col2)
        proto_matrix.append(col2)
      
    return proto_matrix
 

In [283]:
ob = search_google_DF(posts['text'][400:410])
cosineSim(ob)
align_scores(ob)

In [282]:
proto_matrix = append_features(ob)
FeatureMatrix = np.matrix(proto_matrix)
scaler = StandardScaler()
scaled_matrix = scaler.fit_transform(FeatureMatrix)
scaled_matrix

[4.        2.        3.        1.        2.        2.        1.
 1.        0.        0.        0.9615935]


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

[4.        2.        3.        1.        2.        2.        1.
 1.        0.        0.        0.9615935]


In [281]:
scaled_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [259]:
#ML model 
#load model
infile = open('BestModel','rb')
model = pickle.load(infile, encoding='bytes')

In [274]:
#make prediction
y = model.predict(scaled_matrix)
y

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])